In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Definition du path
path = "C:/Users/pterron/Documents/projet_python/Application_ml/Depot_data/"

In [ ]:
#head set up
pd.set_option('display.max_columns', None)

#Import de toutes les datas
df = pd.read_csv(filepath_or_buffer= path + "valeursfoncieres-2018.txt", sep="|", decimal=",", nrows=10000)

#Visu
df.head()
#df.info()

Traitement des données

In [ ]:
#Valeur manquante
values = df.isna().sum()/len(df)
#Colonne vides
df = df.dropna(axis = 1, how='all')

In [ ]:
df.head()

In [ ]:
#Colonnes sous un certain pourcentage
perc = len(df)*0.20
#print(perc)
#print(df.isna().sum())
df = df.dropna(axis = 1, thresh=perc)
#df.isna().sum()

In [ ]:
df.head()

In [ ]:
#Supprimer ligne ou valeur cible nulle est null
df = df.dropna(axis =0, subset='Valeur fonciere')

In [ ]:
#Supprimer les doublons pour supprimer les dependances
df['discretisation'] = df['Date mutation']+df['Code voie']+ df['Voie']+ df['Commune']
df = df.drop_duplicates(subset = 'discretisation', keep=False)
df = df.drop(labels='discretisation', axis = 1)

In [ ]:
df.columns

In [ ]:
#Feature a garder, trie manuel (Valeurs doublons)
l_corr =['Date mutation', 'Nature mutation', 'Valeur fonciere', 'Code postal', 'Code departement', 'Code commune', 'Type local', 'Surface reelle bati', 'Nature culture', 'Surface terrain']
df = df[l_corr]
df.head()

In [ ]:
df.isnull().sum()/len(df) *100

Remplacement des valeurs NA

In [ ]:
df.describe()

In [ ]:
#Correlation
l_corr = ['Date mutation', 'Nature mutation', 'Valeur fonciere', 'Code postal', 'Code commune', 'Type local', 'Surface reelle bati', 'Nature culture', 'Surface terrain']
corr = df[l_corr].corr(numeric_only=True)
corr
#Visualisation
sns.heatmap(corr, annot = True, cmap = 'crest')

#pairplot
#sns.scatterplot(df, x='Date mutation', y='Valeur fonciere' )

In [ ]:
#df.loc[df['Date mutation'] == '09/02/2018']
#df.describe(include='all')
#df['discretisation'].value_counts()
df.head()

## Recuperation de Data open source

### Latitude longitude commune

In [73]:
#Import csv open source (latitude, longitude, prix au m²)
d_comm = pd.read_csv(filepath_or_buffer= path + "communes-departement-region.csv", sep=",", decimal=".")

#Transtypage


#Visu
d_comm.describe(include='all')

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
count,39201,39201,39201.000000,39201,3257,38932.000000,38932.000000,39194.000000,2580,39201,39201,39194,38934,38934.000000,38934
unique,36013,33527,NaN,33675,3042,NaN,NaN,NaN,6,33440,33610,104,101,NaN,18
top,14371,LIVAROT PAYS D AUGE,NaN,MARSEILLE,MOOREA MAIAO,NaN,NaN,NaN,La,Livarot-Pays-d'Auge,Livarot-Pays-d'Auge,62,Pas-de-Calais,NaN,Grand Est
freq,22,22,NaN,27,20,NaN,NaN,NaN,1200,22,22,915,915,NaN,5673
mean,NaN,NaN,46807.021275,NaN,NaN,46.700016,2.756862,251.960606,NaN,NaN,NaN,NaN,NaN,52.116274,NaN
std,NaN,NaN,25902.437435,NaN,NaN,4.484531,4.932593,184.929582,NaN,NaN,NaN,NaN,NaN,24.394799,NaN
min,NaN,NaN,1000.000000,NaN,NaN,-21.340107,-61.779579,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
25%,NaN,NaN,24800.000000,NaN,NaN,45.132322,0.666244,103.000000,NaN,NaN,NaN,NaN,NaN,28.000000,NaN
50%,NaN,NaN,49360.000000,NaN,NaN,47.380915,2.687060,216.000000,NaN,NaN,NaN,NaN,NaN,44.000000,NaN
75%,NaN,NaN,67350.000000,NaN,NaN,48.821741,4.981116,363.000000,NaN,NaN,NaN,NaN,NaN,76.000000,NaN


In [ ]:
d_comm.shape

In [75]:
d_comm['code_postal'] = d_comm['code_postal'].astype(int)
d_comm.code_commune_INSEE.value_counts()

code_commune_INSEE
14371    22
14061    21
98729    20
76618    18
98719    17
         ..
32142     1
32141     1
32140     1
32139     1
99138     1
Name: count, Length: 36013, dtype: int64

In [79]:
#Suppression des doublons
#Code postal
d_comm = d_comm.drop_duplicates(subset=['code_postal'])
#Meme code commune
d_comm = d_comm.drop_duplicates(subset=['code_commune_INSEE'])

d_comm.code_postal.value_counts()
d_comm.code_commune_INSEE.value_counts()

code_commune_INSEE
1001     1
65059    1
65125    1
65100    1
65099    1
        ..
35005    1
35004    1
35003    1
35002    1
99138    1
Name: count, Length: 6138, dtype: int64

In [78]:
d_fil = d_comm.loc[d_comm['code_commune_INSEE'] == '97415']
d_fil

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
38873,97415,ST PAUL,97411,ST PAUL,BOIS DE NEFLES ST PAUL,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion
38874,97415,ST PAUL,97422,ST PAUL,LA SALINE,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion
38875,97415,ST PAUL,97423,ST PAUL,LE GUILLAUME,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion
38876,97415,ST PAUL,97434,ST PAUL,LA SALINE LES BAINS,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion
38878,97415,ST PAUL,97435,ST PAUL,BERNICA,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion
38881,97415,ST PAUL,97460,ST PAUL,NaN,-21.044532,55.322333,15.0,NaN,Saint-Paul,Saint-Paul,974,La Réunion,4.0,La Réunion


In [ ]:
d_comm.shape

### Prix au metre carré ATTENTION, BASE SUR PEU DE VENTE, PEUT ETRE PAS PERTINENT

In [83]:
#Import csv open source (latitude, longitude, prix au m²)
d_prix_mcarré = pd.read_csv(filepath_or_buffer= path + "dvf-communes-2019.csv", sep=";")

#Transtypage

#Visu
d_prix_mcarré.describe(include='all')

,ID,INSEE_COM,INSEE_DEP,INSEE_REG,CODE_EPCI,NOM_COM_M,POPULATION,Nb_Ventes,PrixMoyen_M2
count,34886,34886,34886,34886.000000,3.483700e+04,34886,3.488600e+04,26822.000000,26822.000000
unique,34886,34886,96,NaN,NaN,32432,NaN,NaN,NaN
top,COMMUNE_0000000009760754,32216,62,NaN,NaN,SAINTE COLOMBE,NaN,NaN,NaN
freq,1,1,890,NaN,NaN,12,NaN,NaN,NaN
mean,NaN,NaN,NaN,52.462420,2.127412e+08,NaN,1.950264e+03,24.576243,1635.830177
std,NaN,NaN,NaN,24.216652,2.013492e+07,NaN,1.540069e+04,128.963903,864.263847
min,NaN,NaN,NaN,11.000000,2.000002e+08,NaN,0.000000e+00,1.000000,350.000000
25%,NaN,NaN,NaN,28.000000,2.000659e+08,NaN,1.990000e+02,2.000000,1101.000000
50%,NaN,NaN,NaN,44.000000,2.000696e+08,NaN,4.540000e+02,5.000000,1449.500000
75%,NaN,NaN,NaN,76.000000,2.412007e+08,NaN,1.147000e+03,13.000000,1933.000000


In [74]:
d_prix_mcarré.INSEE_COM.value_counts()

INSEE_COM
32216    1
8491     1
27423    1
70245    1
51298    1
        ..
71493    1
88167    1
13119    1
70226    1
31213    1
Name: count, Length: 34886, dtype: int64

In [72]:
d_filter = d_prix_mcarré.loc[d_prix_mcarré['INSEE_DEP'] == '62']
d_filter

,ID,INSEE_COM,INSEE_DEP,INSEE_REG,CODE_EPCI,NOM_COM_M,POPULATION,Nb_Ventes,PrixMoyen_M2
4,COMMUNE_0000000009727903,62890,62,32,200044030.0,WILLEMAN,182,3.0,967.0
34,COMMUNE_0000000009728493,62840,62,32,200035442.0,VELU,136,1.0,714.0
42,COMMUNE_0000000009727899,62396,62,32,200069672.0,GUINECOURT,18,NaN,NaN
103,COMMUNE_0000000009727696,62274,62,32,246200299.0,DOURGES,5828,28.0,1383.0
122,COMMUNE_0000000009727225,62758,62,32,246200729.0,SAINT MARTIN BOULOGNE,11320,93.0,1707.0
...,...,...,...,...,...,...,...,...,...
34788,COMMUNE_0000000009727845,62892,62,32,200033579.0,WILLERVAL,649,4.0,1572.0
34813,COMMUNE_0000000009728217,62874,62,32,200069482.0,WANQUETIN,720,10.0,1316.0
34841,COMMUNE_0000000009728539,62640,62,32,200069482.0,ORVILLE,403,6.0,904.0
34863,COMMUNE_0000000009727193,62902,62,32,200069037.0,WIZERNES,3317,33.0,1345.0


ATTENTION AUX NOMBRES DE VENTES, IMPACT SUR LE PRIX MOYEN

In [ ]:
l_drop = ['INSEE_COM','INSEE_REG', 'NOM_COM_M', 'PrixMoyen_M2']
d_prix_mcarré.drop()

### Fusion des dataset

In [ ]:
#Ajout des prix au mcarré pour les communes
#merged_df = pd.merge(df1, df2, on='clé', how='inner')

# Machine learning

## Random forest

# DASH

In [ ]:
#Import Packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#Incorporate data

#Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

#App layout
app.layout = html.Div([
    html.Div(className='row', children = 'My first app with Data & Gaphe', 
             style = {'textAlign': 'center', 'color': 'white', 'fontSize': 30}),
    
    html.Div(className='row', children =[
        dcc.RadioItems( options=['Code postal', 'Nature mutation', 'Surface terrain'],
                       value='Code postal',
                       inline=True,
                       id='controls-and-radio-item')],
                       style = {'color': 'white', 'fontSize': 20}),

    html.Div(className='row', children =[
        html.Div(className='six columns', children=[
            dash_table.DataTable(data=df.to_dict('records'), page_size=11, style_table={'overflowX': 'auto'})
        ]),
        html.Div(className='six columns', children=[
            dcc.Graph(figure = {}, id='controls-and-graph')
        ])
    ])     
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    fig = px.histogram(df, x=col_chosen, y='Valeur fonciere', histfunc='avg')
    return fig


#Run the app
if __name__ == '__main__':
    app.run(debug = True)